In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pytensor
import pytensor.tensor as pt
import math

from numba import njit
from pymc.ode import DifferentialEquation
from pytensor.compile.ops import as_op
from scipy.integrate import odeint
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from scipy import stats

from sklearn import linear_model
from numpy.linalg import norm

from numpy import dot
from numpy.linalg import norm

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(10)

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
min_theta_alg = [np.array([0]*13)]*20
method_used_alg = ["None"] * 20
min_distance_alg = [np.inf]*20
min_cosiner_alg = [0]*20
min_cosineb_alg = [0]*20
min_pearsonr_alg = [0]*20
min_pearsonb_alg = [0]*20

min_cosiner_clv = [0]*20
min_cosineb_clv = [0]*20
min_pearsonr_clv = [0]*20
min_pearsonb_clv = [0]*20


min_cosiner_glva = [0]*20
min_cosineb_glva = [0]*20
min_pearsonr_glva = [0]*20
min_pearsonb_glva = [0]*20

min_cosiner_glvr = [0]*20
min_cosineb_glvr = [0]*20
min_pearsonr_glvr = [0]*20
min_pearsonb_glvr = [0]*20

min_cosineb_pe = [0]*20
min_pearsonb_pe = [0]*20

min_cosineb_sp = [0]*20
min_pearsonb_sp = [0]*20



for rep in range(20):
    random.seed(10)
    true_para = np.array([0.31, -0.6, 0.29, -0.01, 0.011, 0.009, -0.01, -0.012, 0.015, 0.3, 0.5, 0.2, 100])
    #true_para = np.array([0.21, -0.4, 0.19, -0.02, 0.016, 0.01, -0.014, -0.017, 0.02, 0.3, 0.5, 0.2, 100])
    time_seg = 1
    cur_time = np.arange(0, 15+time_seg, time_seg)
    #future_time = np.arange(0, 20+0.5, 0.5)
    num_point = len(cur_time)
    num_iteration = 100
    num_run = 20
    n0_est = 200
    noise_scale = 0.05

    distance_sq = [None] * num_run
    distance_trf = [None] * num_run
    distance_lm = [None] * num_run
    cosiner_sq = [None] * num_run
    cosineb_sq = [None] * num_run
    cosiner_trf = [None] * num_run
    cosineb_trf = [None] * num_run
    cosiner_lm = [None] * num_run
    cosineb_lm = [None] * num_run

    # define the right hand side of the ODE equations in the Scipy odeint signature
    from numba import njit


    @njit
    def rhs(X, t, theta):
        # unpack parameters
        x, y, z, n = X
        r1, r2, r3, b12, b13, b21, b23, b31, b32, x0, y0, z0, n0 = theta
        # equations
        dn_dt = r1*x*n + b12*x*y*n*n + b13*x*z*n*n + r2*y*n + b21*x*y*n*n + b23*y*z*n*n + r3*z*n + b31*x*z*n*n + b32*y*z*n*n
        dx_dt = (r1*x*n + b12*x*y*n*n + b13*x*z*n*n - x*dn_dt)/n
        dy_dt = (r2*y*n + b21*x*y*n*n + b23*y*z*n*n - y*dn_dt)/n
        dz_dt = (r3*z*n + b31*x*z*n*n + b32*y*z*n*n - z*dn_dt)/n
        return [dx_dt, dy_dt, dz_dt, dn_dt]

    # note theta = alpha, beta, gamma, delta, xt0, yt0
    theta = true_para
    time = cur_time

    # call Scipy's odeint function, 
    x_y = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))

    random.seed(10)
    n1_noisy_abs = np.maximum(random.uniform(0.0000001, 0.00001), np.array(x_y[:, 0])*np.array(x_y[:, 3])*(1 + noise_scale*np.random.normal(0, 1, len(np.array(x_y[:, 0])))))
    n2_noisy_abs = np.maximum(random.uniform(0.0000001, 0.00001), np.array(x_y[:, 1])*np.array(x_y[:, 3])*(1 + noise_scale*np.random.normal(0, 1, len(np.array(x_y[:, 1])))))
    n3_noisy_abs = np.maximum(random.uniform(0.0000001, 0.00001), np.array(x_y[:, 2])*np.array(x_y[:, 3])*(1 + noise_scale*np.random.normal(0, 1, len(np.array(x_y[:, 2])))))
    abs_abundance_sum = n1_noisy_abs + n2_noisy_abs + n3_noisy_abs
    
    real_value = pd.DataFrame(dict(
        n1_relative = n1_noisy_abs/abs_abundance_sum,
        n2_relative = n2_noisy_abs/abs_abundance_sum,
        n3_relative = n3_noisy_abs/abs_abundance_sum,
        abs_sum = abs_abundance_sum
        ))

    min_distance = np.inf
    min_theta = true_para
    method_used = "None"

    for k in range (num_run):
        
        from numba import njit


        @njit
        def rhs(X, t, theta):
            # unpack parameters
            x, y, z, n = X
            r1, r2, r3, b12, b13, b21, b23, b31, b32, x0, y0, z0, n0 = theta
            # equations
            dn_dt = r1*x*n + b12*x*y*n*n + b13*x*z*n*n + r2*y*n + b21*x*y*n*n + b23*y*z*n*n + r3*z*n + b31*x*z*n*n + b32*y*z*n*n
            dx_dt = (r1*x*n + b12*x*y*n*n + b13*x*z*n*n - x*dn_dt)/n
            dy_dt = (r2*y*n + b21*x*y*n*n + b23*y*z*n*n - y*dn_dt)/n
            dz_dt = (r3*z*n + b31*x*z*n*n + b32*y*z*n*n - z*dn_dt)/n
            return [dx_dt, dy_dt, dz_dt, dn_dt]


        data = pd.DataFrame(dict(
        n1_relative = np.array(real_value.iloc[:, 0]),
        n2_relative = np.array(real_value.iloc[:, 1]),
        n3_relative = np.array(real_value.iloc[:, 2]),
        n1_abs = np.array(real_value.iloc[:, 0]) * n0_est,
        n2_abs = np.array(real_value.iloc[:, 1]) * n0_est,
        n3_abs = np.array(real_value.iloc[:, 2]) * n0_est, 
        ))

        for i in range (num_point-1):
            data.loc[num_point+i] = (1/time_seg)*np.log(data.loc[i+1]) - (1/time_seg)*np.log(data.loc[i])


        x = pd.DataFrame(data.loc[0:num_point-2, ["n2_abs", "n3_abs"]])
        y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n1_abs"])

        regr = linear_model.LinearRegression()
        regr.fit(x, y)

        r1_ini = regr.intercept_[0]
        b12_ini = regr.coef_[0,0]
        b13_ini = regr.coef_[0,1]

        x = pd.DataFrame(data.loc[0:num_point-2, ["n1_abs", "n3_abs"]])
        y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n2_abs"])

        regr = linear_model.LinearRegression()
        regr.fit(x, y)

        r2_ini = regr.intercept_[0]
        b21_ini = regr.coef_[0,0]
        b23_ini = regr.coef_[0,1]

        x = pd.DataFrame(data.loc[0:num_point-2, ["n1_abs", "n2_abs"]])
        y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n3_abs"])

        regr = linear_model.LinearRegression()
        regr.fit(x, y)

        r3_ini = regr.intercept_[0]
        b31_ini = regr.coef_[0,0]
        b32_ini = regr.coef_[0,1]

        theta_estimate = np.array([r1_ini, r2_ini, r3_ini, b12_ini, b13_ini, b21_ini, b23_ini, b31_ini, b32_ini, true_para[-4], true_para[-3], true_para[-2], n0_est])


        theta = theta_estimate

        time = cur_time

         # call Scipy's odeint function, 
        x_y_hat = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))
        data = pd.DataFrame(dict(
            n1_relative = np.array(x_y_hat[:, 0]),
            n2_relative = np.array(x_y_hat[:, 1]),
            n3_relative = np.array(x_y_hat[:, 2]),
            absolute_sum = np.array(x_y_hat[:, 3]),
            n1_abs = np.array(real_value.iloc[:, 0]) * np.array(x_y_hat[:, 3]),
            n2_abs = np.array(real_value.iloc[:, 1]) * np.array(x_y_hat[:, 3]),
            n3_abs = np.array(real_value.iloc[:, 2]) * np.array(x_y_hat[:, 3]), 
        ))
        for i in range (num_point-1):
            data.loc[num_point+i] = (1/time_seg)*np.log(data.loc[i+1]) - (1/time_seg)*np.log(data.loc[i])


        NM = num_iteration
        distance = [None] * NM
        distance_min = np.inf
        theta_min = theta_estimate

        for m in range(NM):
            theta = theta_estimate
            time = cur_time

            x_y_hat = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))
            dis_one = (np.array(np.array(real_value.iloc[:, 0]) - x_y_hat[:, 0]))**2
            dis_two = (np.array(np.array(real_value.iloc[:, 1]) - x_y_hat[:, 1]))**2
            dis_three = (np.array(np.array(real_value.iloc[:, 2]) - x_y_hat[:, 2]))**2
            distance[m] = math.sqrt((np.sum(dis_one) + np.sum(dis_two) + np.sum(dis_three))/num_point/3)
            if distance[m] < distance_min:
                distance_min = distance[m]
                theta_min = theta_estimate

            data = pd.DataFrame(dict(
                n1_relative = np.array(x_y_hat[:, 0]),
                n2_relative = np.array(x_y_hat[:, 1]),
                n3_relative = np.array(x_y_hat[:, 2]),
                absolute_sum = np.array(x_y_hat[:, 3]),
                n1_abs = np.array(real_value.iloc[:, 0]) * np.array(x_y_hat[:, 3]),
                n2_abs = np.array(real_value.iloc[:, 1]) * np.array(x_y_hat[:, 3]),
                n3_abs = np.array(real_value.iloc[:, 2]) * np.array(x_y_hat[:, 3]), 
        ))

            for i in range (num_point-1):
                data.loc[num_point+i] = (1/time_seg)*np.log(data.loc[i+1]) - (1/time_seg)*np.log(data.loc[i])  

            x = pd.DataFrame(data.loc[0:num_point-2, ["n2_abs", "n3_abs"]])
            y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n1_abs"])
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            r1_hat = regr.intercept_[0]
            b12_hat = regr.coef_[0,0]
            b13_hat = regr.coef_[0,1]

            x = pd.DataFrame(data.loc[0:num_point-2, ["n1_abs", "n3_abs"]])
            y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n2_abs"])
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            r2_hat = regr.intercept_[0]
            b21_hat = regr.coef_[0,0]
            b23_hat = regr.coef_[0,1]

            x = pd.DataFrame(data.loc[0:num_point-2, ["n1_abs", "n2_abs"]])
            y = pd.DataFrame(data.loc[num_point:2*num_point-2, "n3_abs"])
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            r3_hat = regr.intercept_[0]
            b31_hat = regr.coef_[0,0]
            b32_hat = regr.coef_[0,1]

            theta_estimate = np.array([r1_hat, r2_hat, r3_hat, b12_hat, b13_hat, b21_hat, b23_hat, b31_hat, b32_hat, true_para[-4], true_para[-3], true_para[-2], n0_est])


        data = pd.DataFrame(dict(
            year = cur_time,
            n1_relative = np.array(real_value.iloc[:, 0]),
            n2_relative = np.array(real_value.iloc[:, 1]),
            n3_relative = np.array(real_value.iloc[:, 2]),
        ))

        # function that calculates residuals based on a given theta
        def ode_model_resid(thetab):
            return (
                np.abs(data[["n1_relative", "n2_relative", "n3_relative"]] - np.delete(odeint(func=rhs, y0=thetab[-4:], t=time, args=(thetab,)), -1, axis=1))
            ).values.flatten()

        #min_distance = np.inf
        #min_theta = theta_min
        #method_used = "None"

        try:
            theta_est = theta_min
            results = leastsq(ode_model_resid, x0=theta_est)

            theta = results[0]
            time = cur_time

            # call Scipy's odeint function0.015, 
            x_y = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))


            dis_one = (np.array(np.array(real_value.iloc[:, 0]) - x_y[:, 0]))**2
            dis_two = (np.array(np.array(real_value.iloc[:, 1]) - x_y[:, 1]))**2
            dis_three = (np.array(np.array(real_value.iloc[:, 2]) - x_y[:, 2]))**2
            distance_sq[k] = math.sqrt((np.sum(dis_one) + np.sum(dis_two) + np.sum(dis_three))/num_point/3)
            cosiner_sq[k] = dot(results[0][:3], true_para[:3])/(norm(results[0][:3])*norm(true_para[:3]))
            cosineb_sq[k] = dot(results[0][3:9], true_para[3:9])/(norm(results[0][3:9])*norm(true_para[3:9]))
            if distance_sq[k] < min_distance:
                    min_distance = distance_sq[k]
                    min_theta = results[0]
                    method_used = "sq"
        except ValueError:
            pass


        try:
            theta_est = theta_min
            results = least_squares(ode_model_resid, x0=theta_est, method = 'trf')

            theta = results.x
            time = cur_time

            # call Scipy's odeint function0.015, 
            x_y = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))

            dis_one = (np.array(np.array(real_value.iloc[:, 0]) - x_y[:, 0]))**2
            dis_two = (np.array(np.array(real_value.iloc[:, 1]) - x_y[:, 1]))**2
            dis_three = (np.array(np.array(real_value.iloc[:, 2]) - x_y[:, 2]))**2
            distance_trf[k]= math.sqrt((np.sum(dis_one) + np.sum(dis_two) + np.sum(dis_three))/num_point/3)
            cosiner_trf[k] = dot(results.x[:3], true_para[:3])/(norm(results.x[:3])*norm(true_para[:3]))
            cosineb_trf[k] = dot(results.x[3:9], true_para[3:9])/(norm(results.x[3:9])*norm(true_para[3:9]))
            if distance_trf[k] < min_distance:
                min_distance = distance_trf[k]
                min_theta = results.x
                method_used = "trf"

            theta_est = theta_min
            results = least_squares(ode_model_resid, x0=theta_est, method = 'lm')
        except ValueError:
            pass

        try:

            theta = results.x
            time = cur_time

            # call Scipy's odeint function0.015, 
            x_y = odeint(func=rhs, y0=theta[-4:], t=time, args=(theta,))

            dis_one = (np.array(np.array(real_value.iloc[:, 0]) - x_y[:, 0]))**2
            dis_two = (np.array(np.array(real_value.iloc[:, 1]) - x_y[:, 1]))**2
            dis_three = (np.array(np.array(real_value.iloc[:, 2]) - x_y[:, 2]))**2
            distance_lm[k]= math.sqrt((np.sum(dis_one) + np.sum(dis_two) + np.sum(dis_three))/num_point/3)
            cosiner_lm[k] = dot(results.x[:3], true_para[:3])/(norm(results.x[:3])*norm(true_para[:3]))
            cosineb_lm[k] = dot(results.x[3:9], true_para[3:9])/(norm(results.x[3:9])*norm(true_para[3:9]))
            if distance_lm[k] < min_distance:
                min_distance = distance_lm[k]
                min_theta = results.x
                method_used = "lm"
        except ValueError:
            pass

    min_theta_alg[rep] = min_theta
    method_used_alg[rep] = method_used
    min_distance_alg[rep] = min_distance
    min_cosiner_alg[rep] = dot(min_theta[:3], true_para[:3])/(norm(min_theta[:3])*norm(true_para[:3]))
    min_pearsonr_alg[rep] = stats.pearsonr(min_theta[:3], true_para[:3])
    min_cosineb_alg[rep] = dot(min_theta[3:9], true_para[3:9])/(norm(min_theta[3:9])*norm(true_para[3:9])) 
    min_pearsonb_alg[rep] = stats.pearsonr(min_theta[3:9], true_para[3:9])
    
    
    data_clv = pd.DataFrame(dict(
    n1_relative = np.array(real_value.iloc[:, 0]),
    n2_over_n1 = np.array(real_value.iloc[:, 1])/np.array(real_value.iloc[:, 0]),
    n3_over_n1 = np.array(real_value.iloc[:, 2])/np.array(real_value.iloc[:, 0]),
    ))
    for i in range (num_point-1):
        data_clv.loc[num_point+i] = (1/time_seg)*np.log(data_clv.loc[i+1])- (1/time_seg)*np.log(data_clv.loc[i]) 
        
    df_clv = pd.DataFrame(dict(
    n1_relative = np.array(real_value.iloc[:-1, 0]),
    n2_relative = np.array(real_value.iloc[:-1, 1]),
    n3_relative = np.array(real_value.iloc[:-1, 2]),
    delta_1 = np.array(data_clv.iloc[num_point:, 0]),
    delta_2 = np.array(data_clv.iloc[num_point:, 1]),
    delta_3 = np.array(data_clv.iloc[num_point:, 2]),
    ))
    
    x = pd.DataFrame(dict(
    n1 = np.array(real_value.iloc[:-1, 0]),
    n3 = np.array(real_value.iloc[:-1, 2])
    ))
    y = df_clv[["delta_2"]]
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    r2_clv = regr.intercept_[0]
    b21_clv = regr.coef_[0,0]
    b23_clv = regr.coef_[0,1]
    
    x = pd.DataFrame(dict(
        n1 = np.array(real_value.iloc[:-1, 0]),
        n2 = np.array(real_value.iloc[:-1, 1])
        ))
    y = df_clv[["delta_3"]]
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    r3_clv = regr.intercept_[0]
    b31_clv = regr.coef_[0,0]
    b32_clv = regr.coef_[0,1]

    theta_b_clv = np.array([b21_clv, b23_clv, b31_clv, b32_clv])
    b12, b13, b21, b23, b31, b32 = true_para[3:9]
    true_b_clv = np.array([b21, b23-b13, b31, b32-b12])
    min_cosineb_clv[rep] = dot(theta_b_clv, true_b_clv)/(norm(theta_b_clv)*norm(true_b_clv))
    min_pearsonb_clv[rep] =stats.pearsonr(theta_b_clv, true_b_clv)

    theta_r_clv = np.array([r2_clv, r3_clv])
    r1, r2, r3 = true_para[0:3]
    true_r_clv = np.array([r2-r1, r3-r1])
    min_cosiner_clv[rep] = dot(theta_r_clv, true_r_clv)/(norm(theta_r_clv)*norm(true_r_clv))
    min_pearsonr_clv[rep] =stats.pearsonr(theta_r_clv, true_r_clv)



    data_glvr = pd.DataFrame(dict(
    n1_relative = np.array(real_value.iloc[:, 0]),
    n2_relative = np.array(real_value.iloc[:, 1]),
    n3_relative = np.array(real_value.iloc[:, 2]),
    n1_abs = np.array(real_value.iloc[:, 0]) * 100,
    n2_abs = np.array(real_value.iloc[:, 1]) * 100,
    n3_abs = np.array(real_value.iloc[:, 2]) * 100, 
    ))
    
    for i in range (num_point-1):
        data_glvr.loc[num_point+i] = (1/time_seg)*np.log(data_glvr.loc[i+1]) - (1/time_seg)*np.log(data_glvr.loc[i])
    
    
    x = pd.DataFrame(data_glvr.loc[0:num_point-2, ["n2_abs", "n3_abs"]])
    y = pd.DataFrame(data_glvr.loc[num_point:2*num_point-2, "n1_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r1_glvr = regr.intercept_[0]
    b12_glvr = regr.coef_[0,0]
    b13_glvr = regr.coef_[0,1]
    
    x = pd.DataFrame(data_glvr.loc[0:num_point-2, ["n1_abs", "n3_abs"]])
    y = pd.DataFrame(data_glvr.loc[num_point:2*num_point-2, "n2_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r2_glvr = regr.intercept_[0]
    b21_glvr = regr.coef_[0,0]
    b23_glvr = regr.coef_[0,1]
    
    x = pd.DataFrame(data_glvr.loc[0:num_point-2, ["n1_abs", "n2_abs"]])
    y = pd.DataFrame(data_glvr.loc[num_point:2*num_point-2, "n3_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r3_glvr = regr.intercept_[0]
    b31_glvr = regr.coef_[0,0]
    b32_glvr = regr.coef_[0,1]
    
    theta_b_glvr = np.array([b12_glvr, b13_glvr, b21_glvr, b23_glvr, b31_glvr, b32_glvr])
    theta_r_glvr = np.array([r1_glvr, r2_glvr, r3_glvr])
    true_b_glvr = true_para[3:9]
    true_r_glvr = true_para[0:3]

    min_cosineb_glvr[rep] = dot(theta_b_glvr, true_b_glvr)/(norm(theta_b_glvr)*norm(true_b_glvr))
    min_pearsonb_glvr[rep] =stats.pearsonr(theta_b_glvr, true_b_glvr)
    min_cosiner_glvr[rep] = dot(theta_r_glvr, true_r_glvr)/(norm(theta_r_glvr)*norm(true_r_glvr))
    min_pearsonr_glvr[rep] =stats.pearsonr(theta_r_glvr, true_r_glvr)


    
    data_glva = pd.DataFrame(dict(
    n1_relative = np.array(real_value.iloc[:, 0]),
    n2_relative = np.array(real_value.iloc[:, 1]),
    n3_relative = np.array(real_value.iloc[:, 2]),
    n1_abs = np.array(real_value.iloc[:, 0]) * np.array(real_value.iloc[:, 3]),
    n2_abs = np.array(real_value.iloc[:, 1]) * np.array(real_value.iloc[:, 3]),
    n3_abs = np.array(real_value.iloc[:, 2]) * np.array(real_value.iloc[:, 3]), 
    ))
    
    for i in range (num_point-1):
        data_glva.loc[num_point+i] = (1/time_seg)*np.log(data_glva.loc[i+1]) - (1/time_seg)*np.log(data_glva.loc[i])
    
    
    x = pd.DataFrame(data_glva.loc[0:num_point-2, ["n2_abs", "n3_abs"]])
    y = pd.DataFrame(data_glva.loc[num_point:2*num_point-2, "n1_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r1_glva = regr.intercept_[0]
    b12_glva = regr.coef_[0,0]
    b13_glva = regr.coef_[0,1]
    
    x = pd.DataFrame(data_glva.loc[0:num_point-2, ["n1_abs", "n3_abs"]])
    y = pd.DataFrame(data_glva.loc[num_point:2*num_point-2, "n2_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r2_glva = regr.intercept_[0]
    b21_glva = regr.coef_[0,0]
    b23_glva = regr.coef_[0,1]
    
    x = pd.DataFrame(data_glva.loc[0:num_point-2, ["n1_abs", "n2_abs"]])
    y = pd.DataFrame(data_glva.loc[num_point:2*num_point-2, "n3_abs"])

    regr = linear_model.LinearRegression()
    regr.fit(x, y)

    r3_glva = regr.intercept_[0]
    b31_glva = regr.coef_[0,0]
    b32_glva = regr.coef_[0,1]
    
    theta_b_glva = np.array([b12_glva, b13_glva, b21_glva, b23_glva, b31_glva, b32_glva])
    theta_r_glva = np.array([r1_glva, r2_glva, r3_glva])
    true_b_glva = true_para[3:9]
    true_r_glva = true_para[0:3]

    min_cosineb_glva[rep] = dot(theta_b_glva, true_b_glva)/(norm(theta_b_glva)*norm(true_b_glva))
    min_pearsonb_glva[rep] = stats.pearsonr(theta_b_glva, true_b_glva)
    min_cosiner_glva[rep] = dot(theta_r_glva, true_r_glva)/(norm(theta_r_glva)*norm(true_r_glva))
    min_pearsonr_glva[rep] = stats.pearsonr(theta_r_glva, true_r_glva)


    sp21=sp12=stats.spearmanr(real_value.iloc[:, 0], real_value.iloc[:, 1])[0]
    sp31=sp13=stats.spearmanr(real_value.iloc[:, 0], real_value.iloc[:, 2])[0]
    sp23=sp32=stats.spearmanr(real_value.iloc[:, 1], real_value.iloc[:, 2])[0]

    pe21=pe12=stats.pearsonr(real_value.iloc[:, 0], real_value.iloc[:, 1])[0]
    pe31=pe13=stats.pearsonr(real_value.iloc[:, 0], real_value.iloc[:, 2])[0]
    pe23=pe32=stats.pearsonr(real_value.iloc[:, 1], real_value.iloc[:, 2])[0]

    theta_b_sp = np.array([sp12, sp13, sp21, sp23, sp31, sp32])
    theta_b_pe = np.array([pe12, pe13, pe21, pe23, pe31, pe32])

    true_b_cor = true_para[3:9]
    
    min_cosineb_sp[rep] = dot(theta_b_sp, true_b_cor)/(norm(theta_b_sp)*norm(true_b_cor))
    min_pearsonb_sp[rep] = stats.pearsonr(theta_b_sp, true_b_cor)

    min_cosineb_pe[rep] = dot(theta_b_pe, true_b_cor)/(norm(theta_b_pe)*norm(true_b_cor))
    min_pearsonb_pe[rep] = stats.pearsonr(theta_b_pe, true_b_cor)
    

In [33]:
with open("noise05_ran15.txt", "w", encoding="utf-8") as file:
    file.write("True para: \n")

with open("noise05_ran15.txt", "a", encoding="utf-8") as file:
    np.savetxt(file, true_para, newline='\n',  fmt="%.8g")
    file.write("\nnoise_scale: \n")
    file.write(str(noise_scale))
    file.write("\ncur_time: \n")
    np.savetxt(file, cur_time, newline='\n',  fmt="%.8g")
    file.write("\nmin_theta_alg: \n")
    np.savetxt(file, min_theta_alg, newline='\n',  fmt="%.8g")
    file.write("\nmethod_used_alg: \n")
    np.savetxt(file, method_used_alg, newline='\n', fmt="%s")
    
    file.write("\nmin_distance_alg: \n")
    file.write(f"{np.mean(min_distance_alg):.8g}")
    file.write("±")
    file.write(f"{np.std(min_distance_alg):.8g}\n")
    np.savetxt(file, min_distance_alg, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosineb_alg: \n")
    file.write(f"{np.mean(min_cosineb_alg):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_alg):.8g}\n")
    np.savetxt(file, min_cosineb_alg, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosineb_clv: \n")
    file.write(f"{np.mean(min_cosineb_clv):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_clv):.8g}\n")
    np.savetxt(file, min_cosineb_clv, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosineb_glva: \n")
    file.write(f"{np.mean(min_cosineb_glva):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_glva):.8g}\n")
    np.savetxt(file, min_cosineb_glva, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosineb_glvr: \n")
    file.write(f"{np.mean(min_cosineb_glvr):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_glvr):.8g}\n")
    np.savetxt(file, min_cosineb_glvr, newline='\n', fmt="%.8g")

    #####
    file.write("\nmin_cosineb_pe: \n")
    file.write(f"{np.mean(min_cosineb_pe):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_pe):.8g}\n")
    np.savetxt(file, min_cosineb_pe, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosineb_sp: \n")
    file.write(f"{np.mean(min_cosineb_sp):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosineb_sp):.8g}\n")
    np.savetxt(file, min_cosineb_sp, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_alg: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_alg]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_alg]):.8g}\n")
    np.savetxt(file, min_pearsonb_alg, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_clv: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_clv]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_clv]):.8g}\n")
    np.savetxt(file, min_pearsonb_clv, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_glva: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_glva]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_glva]):.8g}\n")
    np.savetxt(file, min_pearsonb_glva, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_glvr: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_glvr]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_glvr]):.8g}\n")
    np.savetxt(file, min_pearsonb_glvr, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_pe: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_pe]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_pe]):.8g}\n")
    np.savetxt(file, min_pearsonb_pe, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonb_sp: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonb_sp]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonb_sp]):.8g}\n")
    np.savetxt(file, min_pearsonb_sp, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosiner_alg: \n")
    file.write(f"{np.mean(min_cosiner_alg):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosiner_alg):.8g}\n")
    np.savetxt(file, min_cosiner_alg, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosiner_clv: \n")
    file.write(f"{np.mean(min_cosiner_clv):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosiner_clv):.8g}\n")
    np.savetxt(file, min_cosiner_clv, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosiner_glva: \n")
    file.write(f"{np.mean(min_cosiner_glva):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosiner_glva):.8g}\n")
    np.savetxt(file, min_cosiner_glva, newline='\n', fmt="%.8g")
    
    file.write("\nmin_cosiner_glvr: \n")
    file.write(f"{np.mean(min_cosiner_glvr):.8g}")
    file.write("±")
    file.write(f"{np.std(min_cosiner_glvr):.8g}\n")
    np.savetxt(file, min_cosiner_glvr, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonr_alg: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonr_alg]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonr_alg]):.8g}\n")
    np.savetxt(file, min_pearsonr_alg, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonr_glva: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonr_glva]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonr_glva]):.8g}\n")
    np.savetxt(file, min_pearsonr_glva, newline='\n', fmt="%.8g")
    
    file.write("\nmin_pearsonr_glvr: \n")
    file.write(f"{np.mean([t[0] for t in min_pearsonr_glvr]):.8g}")
    file.write("±")
    file.write(f"{np.std([t[0] for t in min_pearsonr_glvr]):.8g}\n")
    np.savetxt(file, min_pearsonr_glvr, newline='\n', fmt="%.8g")
    